In [1]:
import pandas as pd
from collections import defaultdict
infile="/Users/irffanalahi/Research/Research_code/likephased-seq/DownsamplingDMR/downsamples_number/DMR50.txt"

fragsize=170


indf=pd.read_csv(infile,sep="\t")
indf.head()

,DMRchr,DMRstart,DMRend,chrom,start,end,celltype
0,chr1,12168470,12169152,chr1,12168469,12168471,[Tr]
1,chr1,12168470,12169152,chr1,12168485,12168487,[Tr]
2,chr1,12168470,12169152,chr1,12168489,12168491,[Tr]
3,chr1,12168470,12169152,chr1,12168553,12168555,[Tr]
4,chr1,12168470,12169152,chr1,12168794,12168796,[Tr]


In [2]:
celllist=list(set((indf['celltype']).tolist()))
print(celllist)

['[mNeu]', '[Mn]', '[mB]', '[Eo]', '[Mg]', '[m4]', '[NK]', '[CD8]', '[nB]', '[m8]', '[CD4]', '[Tr]', '[Er]', '[DC]']


In [3]:
outdict=defaultdict(list)


grouped=indf.groupby(['DMRchr','DMRstart','DMRend'])

for name, group in grouped:
    outdict['chrom'].append(name[0])
    outdict['start'].append(name[1])
    outdict['end'].append(name[2])
    dlength=int(name[2])-int(name[1])
    outdict['DMRlength'].append(dlength)
    reporter=1.0*dlength/fragsize  ################
    outdict['reporter'].append(reporter)
    
    outdict['#cpg'].append(group.shape[0])
    outdict['celltype'].append((group['celltype'].tolist())[0])

    
    
outdf=pd.DataFrame.from_dict(outdict)

#outdf['cpg_per_reporter']=outdf['reporter']/outdf['#cpg']

outdf.head()

,chrom,start,end,DMRlength,reporter,#cpg,celltype
0,chr1,1221864,1222863,999,5.876471,36,[NK]
1,chr1,2301964,2302282,318,1.870588,10,[CD8]
2,chr1,6063900,6063995,95,0.558824,4,[DC]
3,chr1,6282776,6283095,319,1.876471,8,[Eo]
4,chr1,6498701,6499009,308,1.811765,9,[DC]


In [4]:
summarydict=defaultdict(list)
outdfgr=outdf.groupby(['celltype'])
for name, group in outdfgr:
    summarydict['celltype'].append(name)
    summarydict['total_DMR'].append(group.shape[0])
    summarydict['avg_cpg_per_DMR'].append(group['#cpg'].mean())
    summarydict['total_reporter'].append(group['reporter'].sum())

summarydf=pd.DataFrame.from_dict(summarydict)

summarydf.head()
    

,celltype,total_DMR,avg_cpg_per_DMR,total_reporter
0,[CD4],21,7.904762,29.147059
1,[CD8],26,10.076923,46.676471
2,[DC],50,6.760000,65.800000
3,[Eo],50,9.080000,88.464706
4,[Er],50,8.440000,85.764706


In [5]:
outname=infile+"_stats.txt"
outdf.to_csv(outname,sep="\t",index=False)
summarydf.to_csv(outname+"_summary.txt",sep="\t",index=False)